In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import gc

/kaggle/input/202074fraud/加工后的数据/readme.txt
/kaggle/input/202074fraud/加工后的数据/test_drop_50%ofNaN.csv
/kaggle/input/202074fraud/加工后的数据/test_1000lines.csv
/kaggle/input/202074fraud/加工后的数据/train_drop_50%ofNaN.csv
/kaggle/input/202074fraud/加工后的数据/train_identity_1000lines.csv
/kaggle/input/202074fraud/加工后的数据/train_transaction_1000lines.csv
/kaggle/input/202074fraud/加工后的数据/test.csv
/kaggle/input/202074fraud/加工后的数据/train_1000lines.csv
/kaggle/input/202074fraud/加工后的数据/train.csv


In [2]:
mpath = '/kaggle/input/202074fraud/加工后的数据/'
train = pd.read_csv(f'{mpath}train.csv')
test = pd.read_csv(f'{mpath}test.csv')

### 将以下变量独热化

#### train

In [ ]:
need_dummy_cols = []
print('train中是object的列有:\tdrop_duplicates后剩下的个数')
for x in train.columns:
    if train[x].dtype=='O':
        print('{:30}'.format(x),end='\t')
        print(len(train[x].drop_duplicates()))
        need_dummy_cols.append(x)

##### test

In [ ]:
need_dummy_cols_test = []
print('test中是object的列有:\tdrop_duplicates后剩下的个数')
for x in test.columns:
    if test[x].dtype=='O':
        print('{:30}'.format(x),end='\t')
        print(len(test[x].drop_duplicates()))
        need_dummy_cols_test.append(x)

##### train和test一起看

In [ ]:
print('是object的列有:\t train表格drop_duplicates后剩下的个数\ttest表格drop_duplicates后剩下的个数')
for x,y in zip(need_dummy_cols,need_dummy_cols_test):
    print('{:30}'.format(x),end='\t')
    print(len(train[x].drop_duplicates()),end='\t\t\t')
    print('{:30}'.format(y),end='\t')
    print(len(test[y].drop_duplicates()))

#### 查看P_emaildomain和R_emaildomain

In [ ]:
for x in list(test['P_emaildomain'].drop_duplicates()):
    if x in list(train['P_emaildomain'].drop_duplicates()):
        print('{} in train!'.format(x))
    else:
        print('{} not in train!'.format(x))
        print('-'*20)

In [ ]:
for x in list(test['R_emaildomain'].drop_duplicates()):
    if x in list(train['R_emaildomain'].drop_duplicates()):
        print('{} in train!'.format(x))
    else:
        print('{} not in train!'.format(x))
        print('-'*20)

In [3]:
emails = {'gmail': 'google', 'att.net': 'att', 'twc.com': 'spectrum', 
          'scranton.edu': 'other', 'optonline.net': 'other', 'hotmail.co.uk': 'microsoft',
          'comcast.net': 'other', 'yahoo.com.mx': 'yahoo', 'yahoo.fr': 'yahoo',
          'yahoo.es': 'yahoo', 'charter.net': 'spectrum', 'live.com': 'microsoft', 
          'aim.com': 'aol', 'hotmail.de': 'microsoft', 'centurylink.net': 'centurylink',
          'gmail.com': 'google', 'me.com': 'apple', 'earthlink.net': 'other', 'gmx.de': 'other',
          'web.de': 'other', 'cfl.rr.com': 'other', 'hotmail.com': 'microsoft', 
          'protonmail.com': 'other', 'hotmail.fr': 'microsoft', 'windstream.net': 'other', 
          'outlook.es': 'microsoft', 'yahoo.co.jp': 'yahoo', 'yahoo.de': 'yahoo',
          'servicios-ta.com': 'other', 'netzero.net': 'other', 'suddenlink.net': 'other',
          'roadrunner.com': 'other', 'sc.rr.com': 'other', 'live.fr': 'microsoft',
          'verizon.net': 'yahoo', 'msn.com': 'microsoft', 'q.com': 'centurylink', 
          'prodigy.net.mx': 'att', 'frontier.com': 'yahoo', 'anonymous.com': 'other', 
          'rocketmail.com': 'yahoo', 'sbcglobal.net': 'att', 'frontiernet.net': 'yahoo', 
          'ymail.com': 'yahoo', 'outlook.com': 'microsoft', 'mail.com': 'other', 
          'bellsouth.net': 'other', 'embarqmail.com': 'centurylink', 'cableone.net': 'other', 
          'hotmail.es': 'microsoft', 'mac.com': 'apple', 'yahoo.co.uk': 'yahoo', 'netzero.com': 'other', 
          'yahoo.com': 'yahoo', 'live.com.mx': 'microsoft', 'ptd.net': 'other', 'cox.net': 'other',
          'aol.com': 'aol', 'juno.com': 'other', 'icloud.com': 'apple'}
us_emails = ['gmail', 'net', 'edu']

# https://www.kaggle.com/c/ieee-fraud-detection/discussion/100499#latest-579654
df_train = train
df_test = test
for c in ['P_emaildomain', 'R_emaildomain']:
    df_train[c + '_bin'] = df_train[c].map(emails)
    df_test[c + '_bin'] = df_test[c].map(emails)
    
    df_train[c + '_suffix'] = df_train[c].map(lambda x: str(x).split('.')[-1])
    df_test[c + '_suffix'] = df_test[c].map(lambda x: str(x).split('.')[-1])
    
    df_train[c + '_suffix'] = df_train[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')
    df_test[c + '_suffix'] = df_test[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')

In [4]:
train.drop(columns=['P_emaildomain', 'R_emaildomain'],inplace=True)
test.drop(columns=['P_emaildomain', 'R_emaildomain'],inplace=True)

##### train和test在id列上-符号的位置不匹配，需要注意这一点

In [ ]:
for x,y in zip(need_dummy_cols,need_dummy_cols_test):
    if x==y:
        print('ok   ,\tx:{:15}\ty:{:15}'.format(x,y))
    else:
        print('false,\tx:{:15}\ty:{:15}'.format(x,y))

## 修改test的id-1到id_1

In [8]:
id_col_can = ['id_{:02}'.format(x) for x in range(1,39)]
id_col_can_old = ['id-{:02}'.format(x) for x in range(1,39)]
correct_dict = {}
for new,old in zip(id_col_can,id_col_can_old):
    correct_dict[old] = new
test.rename(columns=correct_dict,inplace=True)

#### 需要提取元素的几列 id-30、id_31、id_33、DeviceInfo

In [ ]:
for x in list(test['id_30'].drop_duplicates()):
    if x in list(train['id_30'].drop_duplicates()):
        print('{} in train!'.format(x))
    else:
        print('{} not in train!'.format(x))
        print('-'*20)

In [ ]:
for x in list(test['id_31'].drop_duplicates()):
    if x in list(train['id_31'].drop_duplicates()):
        print('{} in train!'.format(x))
    else:
        print('{} not in train!'.format(x))
        print('-'*20)

In [ ]:
for x in list(test['id_33'].drop_duplicates()):
    if x in list(train['id_33'].drop_duplicates()):
        print('{} in train!'.format(x))
    else:
        print('{} not in train!'.format(x))
        print('-'*20)

In [ ]:
for x in list(test['DeviceInfo'].drop_duplicates()):
    if x in list(train['DeviceInfo'].drop_duplicates()):
        print('{} in train!'.format(x))
    else:
        print('{} not in train!'.format(x))
        print('-'*20)

In [ ]:
test['DeviceInfo'].drop_duplicates()

##### 处理方法用split分割，提取第一个特征即可

#### id_30

In [ ]:
train['id_30'].drop_duplicates()

In [10]:
id_30_new = pd.DataFrame(['0' for x in range(train.shape[0])],columns=['id_30_new'],dtype=object)
for i,x in enumerate(train['id_30']):
    try:
        id_30_new.iloc[i,0] = (str(x.split()[0]))
    except:
        id_30_new.iloc[i,0] = np.nan
######-------######
id_30_new_test = pd.DataFrame(['0' for x in range(test.shape[0])],columns=['id_30_new'],dtype=object)
for i,x in enumerate(test['id_30']):
    try:
        id_30_new_test.iloc[i,0] = (str(x.split()[0]))
    except:
        id_30_new_test.iloc[i,0] = np.nan


train['id_30_new'] = id_30_new
test['id_30_new'] = id_30_new_test

train.drop(columns=['id_30'],inplace=True)
test.drop(columns=['id_30'],inplace=True)



In [9]:
test['id_30_new'].drop_duplicates()

0             NaN
39        Android
73            iOS
128       Windows
159           Mac
245         Linux
29344        func
355197      other
Name: id_30_new, dtype: object

In [10]:
train['id_30_new'].drop_duplicates()

0            NaN
4        Android
8            iOS
16           Mac
17       Windows
69         Linux
18582       func
33565      other
Name: id_30_new, dtype: object

#### id_31

### 由于id_31训练集和测试集的差别太大，故直接舍弃该列

In [11]:
id_31_new = pd.DataFrame(['0' for x in range(train.shape[0])],columns=['id_31_new'],dtype=object) #2个地方要改
for i,x in enumerate(train['id_31']): #1个
    try:
        id_31_new.iloc[i,0] = (str(x.split()[0])) #1个
    except:
        id_31_new.iloc[i,0] = np.nan #1个
######-------######
id_31_new_test = pd.DataFrame(['0' for x in range(test.shape[0])],columns=['id_31_new'],dtype=object) #2
for i,x in enumerate(test['id_31']):
    try:
        id_31_new_test.iloc[i,0] = (str(x.split()[0])) #1
    except:
        id_31_new_test.iloc[i,0] = np.nan #1


train['id_31_new'] = id_31_new #2
test['id_31_new'] = id_31_new_test #2

train.drop(columns=['id_31'],inplace=True) #1
test.drop(columns=['id_31'],inplace=True) #1

train.drop(columns=['id_31_new'],inplace=True) #1
test.drop(columns=['id_31_new'],inplace=True) #1


In [ ]:
train['id_31_new'].drop_duplicates()

In [ ]:
test['id_31_new'].drop_duplicates()

In [ ]:
for x in list(test['id_31_new'].drop_duplicates()):
    if x in list(train['id_31_new'].drop_duplicates()):
        print('yes')
    else:
        print('no')

#### id_33

### id_33的train和test差别太大，删掉该列

In [12]:
id_33_new = pd.DataFrame(['0' for x in range(train.shape[0])],columns=['id_33_new'],dtype=object) #2个地方要改
for i,x in enumerate(train['id_33']): #1个
    try:
        id_33_new.iloc[i,0] = (str(x.split()[0])) #1个
    except:
        id_33_new.iloc[i,0] = np.nan #1个
######-------######
id_33_new_test = pd.DataFrame(['0' for x in range(test.shape[0])],columns=['id_33_new'],dtype=object) #2
for i,x in enumerate(test['id_33']):
    try:
        id_33_new_test.iloc[i,0] = (str(x.split()[0])) #1
    except:
        id_33_new_test.iloc[i,0] = np.nan #1


train['id_33_new'] = id_33_new #2
test['id_33_new'] = id_33_new_test #2

train.drop(columns=['id_33'],inplace=True) #1
test.drop(columns=['id_33'],inplace=True) #1

train.drop(columns=['id_33_new'],inplace=True) #1
test.drop(columns=['id_33_new'],inplace=True) #1

In [ ]:
train['id_33_new'].drop_duplicates()

In [ ]:
test['id_33_new'].drop_duplicates()

#### DeviceInfo

### DeviceInfo的train和test差别太大，删掉该列

In [13]:
DeviceInfo_new = pd.DataFrame(['0' for x in range(train.shape[0])],columns=['DeviceInfo_new'],dtype=object) #2个地方要改
for i,x in enumerate(train['DeviceInfo']): #1个
    try:
        DeviceInfo_new.iloc[i,0] = (str(x.split()[0])) #1个
    except:
        DeviceInfo_new.iloc[i,0] = np.nan #1个
######-------######
DeviceInfo_new_test = pd.DataFrame(['0' for x in range(test.shape[0])],columns=['DeviceInfo_new'],dtype=object) #2
for i,x in enumerate(test['DeviceInfo']):
    try:
        DeviceInfo_new_test.iloc[i,0] = (str(x.split()[0])) #1
    except:
        DeviceInfo_new_test.iloc[i,0] = np.nan #1


train['DeviceInfo_new'] = DeviceInfo_new #2
test['DeviceInfo_new'] = DeviceInfo_new_test #2

train.drop(columns=['DeviceInfo'],inplace=True) #1
test.drop(columns=['DeviceInfo'],inplace=True) #1

train.drop(columns=['DeviceInfo_new'],inplace=True) #1
test.drop(columns=['DeviceInfo_new'],inplace=True) #1

In [ ]:
train['DeviceInfo_new'].drop_duplicates()

In [ ]:
test['DeviceInfo_new'].drop_duplicates()

In [ ]:
list(train.columns)

## 将object型数据转化为独热向量

In [14]:
need_dummy_cols = []
print('train中是object的列有:\tdrop_duplicates后剩下的个数')
for x in train.columns:
    if train[x].dtype=='O':
        print('{:30}'.format(x),end='\t')
        print(len(train[x].drop_duplicates()))
        need_dummy_cols.append(x)

need_dummy_cols_test = []
print('test中是object的列有:\tdrop_duplicates后剩下的个数')
for x in test.columns:
    if test[x].dtype=='O':
        print('{:30}'.format(x),end='\t')
        print(len(test[x].drop_duplicates()))
        need_dummy_cols_test.append(x)

train中是object的列有:	drop_duplicates后剩下的个数
ProductCD                     	5
card4                         	5
card6                         	5
M1                            	3
M2                            	3
M3                            	3
M4                            	4
M5                            	3
M6                            	3
M7                            	3
M8                            	3
M9                            	3
id_12                         	3
id_15                         	4
id_16                         	3
id_23                         	4
id_27                         	3
id_28                         	3
id_29                         	3
id_34                         	5
id_35                         	3
id_36                         	3
id_37                         	3
id_38                         	3
DeviceType                    	3
P_emaildomain_bin             	10
P_emaildomain_suffix          	9
R_emaildomain_bin             	10
R_emaildomain_suffix          	9
i

In [15]:
set(need_dummy_cols) == set(need_dummy_cols_test)

True

In [16]:
print(train.shape)
print(test.shape)

(590540, 433)
(506691, 432)


In [17]:
train_and_test = pd.concat([train,test])

In [39]:
del train,test

In [30]:
gc.collect()

20

In [35]:
train_and_test.iloc[:590540,:] #train

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_35,id_36,id_37,id_38,DeviceType,P_emaildomain_bin,P_emaildomain_suffix,R_emaildomain_bin,R_emaildomain_suffix,id_30_new
0,2987000,0.0,86400,68.50,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,NaN
1,2987001,0.0,86401,29.00,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,google,com,NaN,nan,NaN
2,2987002,0.0,86469,59.00,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,microsoft,com,NaN,nan,NaN
3,2987003,0.0,86499,50.00,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,yahoo,com,NaN,nan,NaN
4,2987004,0.0,86506,50.00,H,4497,514.0,150.0,mastercard,102.0,...,T,F,T,T,mobile,google,com,NaN,nan,Android
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535,0.0,15811047,49.00,W,6550,NaN,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,NaN
590536,3577536,0.0,15811049,39.50,W,10444,225.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,google,com,NaN,nan,NaN
590537,3577537,0.0,15811079,30.95,W,12037,595.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,google,com,NaN,nan,NaN
590538,3577538,0.0,15811088,117.00,W,7826,481.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,aol,com,NaN,nan,NaN


In [37]:
train_and_test.iloc[590540:,:].drop(columns=['isFraud']) #test

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id_35,id_36,id_37,id_38,DeviceType,P_emaildomain_bin,P_emaildomain_suffix,R_emaildomain_bin,R_emaildomain_suffix,id_30_new
0,3663549,18403224,31.950,W,10409,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,google,com,NaN,nan,NaN
1,3663550,18403263,49.000,W,4272,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,aol,com,NaN,nan,NaN
2,3663551,18403310,171.000,W,4476,574.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,microsoft,com,NaN,nan,NaN
3,3663552,18403310,284.950,W,10989,360.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,google,com,NaN,nan,NaN
4,3663553,18403317,67.950,W,18018,452.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,google,com,NaN,nan,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506686,4170235,34214279,94.679,C,13832,375.0,185.0,mastercard,224.0,debit,...,NaN,NaN,NaN,NaN,NaN,google,com,google,com,NaN
506687,4170236,34214287,12.173,C,3154,408.0,185.0,mastercard,224.0,debit,...,F,F,T,F,mobile,microsoft,com,microsoft,com,NaN
506688,4170237,34214326,49.000,W,16661,490.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,microsoft,com,NaN,nan,NaN
506689,4170238,34214337,202.000,W,16621,516.0,150.0,mastercard,224.0,debit,...,NaN,NaN,NaN,NaN,NaN,microsoft,com,NaN,nan,NaN


In [19]:
train_and_test = pd.get_dummies(train_and_test,prefix=need_dummy_cols)

In [22]:
train_and_test.iloc[:590540,:].to_csv('train_dummy_with_nan.csv',index=False)

In [24]:
train_and_test.iloc[590540:,:].drop(columns=['isFraud']).to_csv('test_dummy_with_nan.csv',index=False)

## 用零来填缺失值